In [7]:
import os
import multiprocessing

# os.environ["XLA_FLAGS"] = "--xla_force_host_platform_device_count={}".format(
#     multiprocessing.cpu_count()
# )

import jax
import jax.numpy as jnp


from qbism import sic_povm
import tomllib
from qdots_qll.exp_design import (
    RandomExpDesign,
    MaxDetFimExpDesign,
    MaxTraceFimExpDesign,
)
from qdots_qll.run import Run, initial_run_from_config
from qdots_qll.smc import SMCUpdater, SMC_run
from qdots_qll.resamplers import LWResampler
from qdots_qll.stop_conditions import TerminationChecker

# from qdots_qll.models.game import true_pars
from qdots_qll.distributions import (
    est_cov,
    est_mean,
    initialize_particle_locations,
    initialize_weights,
)
from qdots_qll.models.models_scratch_for_drafting import (
    two_qdots_separable_maps,
)
import joblib
import logging
from datetime import datetime
from qdots_qll.utils.generate_initial_state import max_entangled_dm_vec
from pprint import pformat


init_time = datetime.today().strftime("%Y-%m-%d_%H:%M:%S")
run_filename = "results/run_" + init_time

# logging.basicConfig(
#     filename=run_filename + ".log",
#     level=logging.INFO,
#     encoding="utf-8",
#     format="%(asctime)s - - %(levelname)s: %(message)s",
# )


with open("job.toml", "rb") as f:
    config = tomllib.load(f)

# logging.info(pformat(config["run"]))


number_of_runs = config["run"]["number_of_runs"]
number_of_runs_compilation = config["run_for_compilation"]["number_of_runs"]

print(number_of_runs)
print(number_of_runs_compilation)


model = two_qdots_separable_maps(POVM_array=jnp.array(sic_povm(4)))

seed = config["run"]["seed"]


key = jax.random.PRNGKey(seed=seed)


key, subkey = jax.random.split(key)


# print(initial_runs_compilation)
# keys = jax.random.split(key, number_of_runs)
true_pars = jnp.array(config["run"]["true_parameters"])


exp_design_dict = {
    "random": RandomExpDesign(0.01, 40),
    "maxdetfim": MaxDetFimExpDesign(0.01, 40, 20, lr=0.5),
    "maxtracefim": MaxTraceFimExpDesign(0.01, 40, 20, lr=0.5),
}

exp_design = exp_design_dict[config["run"]["exp_design"]]


resampler = LWResampler()


smcupdater = SMCUpdater(
    model=model,
    exp_design=exp_design,
    resampler=resampler,
    initial_state=max_entangled_dm_vec,
    true_pars=true_pars,
    number_exp_repetitions=1,
)


96
5


In [8]:
keys_for_compilation = jax.random.split(subkey, number_of_runs_compilation)

initial_runs_compilation = jax.vmap(
    lambda key: initial_run_from_config(
        key,
        model,
        config["run_for_compilation"],
    )
)(keys_for_compilation)


example_run = (
    lambda key: initial_run_from_config(
        key,
        model,
        config["run_for_compilation"],
    )
)(keys_for_compilation[0])

stopper_for_compilation = TerminationChecker(
    config["run_for_compilation"]["max_iterations"]
)

In [10]:
import jax.tree_util

In [16]:
jax.tree_util.tree_map(lambda run: run + 1, example_run)

Run(
  iteration=1,
  key=u32[2],
  time=1,
  weights=f32[100],
  particles_locations=f32[100,4],
  times_array=f32[5],
  estimates_array=f32[5,4],
  cov_array=f32[5,4,4],
  max_iterations=6,
  min_iterations=2,
  std_threshold=1.00000001
)